In [21]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import torch
from pytorch_datasets.img_dataset import ImageDataset
from torch.utils.data import DataLoader, random_split
from models.vae_img import ImageVAE
from torch.optim import Adam

Using cuda device
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
cuda_device = 0 # set the index of the GPU you want to use
torch.cuda.set_device(cuda_device)
torch.backends.cudnn.benchmark = True

# set the maximum GPU memory usage
max_memory_usage = 0.98 # set the maximum memory usage as a fraction of the available memory
# torch.cuda.set_max_memory_allocated(int(torch.cuda.get_device_properties(cuda_device).total_memory * max_memory_usage))
torch.cuda.set_per_process_memory_fraction(max_memory_usage, cuda_device)

In [16]:
print(torch.cuda.max_memory_allocated(cuda_device)/1024/1024/1024)
torch.cuda.empty_cache()

0.10911703109741211


In [17]:
# Load and clean data

instagram_data = pd.read_csv("data/instagram_data.csv")
data = instagram_data.dropna(subset=["id"]).reset_index(drop=True)
print(len(data))

print(f"Removed {len(instagram_data) - len(data)} rows due to N/A images.")

/tmp/ipykernel_46489/2114043278.py:3: DtypeWarning: Columns (24,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,62,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  instagram_data = pd.read_csv("data/instagram_data.csv")


380987
Removed 0 rows due to N/A images.


In [18]:
# Create Dataset

image_usernames = data["username"].tolist()
image_file_ids = data["id"].tolist()
image_encoding_folder_path = "data/VGG/"
post_classes = data["Party"].tolist()

dataset = ImageDataset(image_usernames, image_file_ids, image_encoding_folder_path, post_classes)

train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("Complete Dataset Size:", len(dataset))
print("Training Dataset Size:", len(train_dataset))
print("Test Dataset Size:", len(test_dataset))

11755 of data removed because matching VGG image embeddings not found.
Complete Dataset Size: 369232
Training Dataset Size: 295386
Test Dataset Size: 73846


In [19]:
# Set hyperparameters

num_epochs = 10
batch_size = 128
learning_rate = 0.001

In [22]:
# Train model

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# small_dataset = ImageDataset(image_usernames[:1000], image_file_ids[:1000], image_encoding_folder_path, post_classes[:1000])
# small_dataloader = DataLoader(small_dataset, batch_size=batch_size, shuffle=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

model = ImageVAE(device, 32, 32, 2048, dataset.num_post_classes).to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0

    for batch_idx, (image_embedding, party) in enumerate(train_dataloader):
        image_embedding = image_embedding.to(device)

        decoded_image, mu, logvar, classifier_result = model(image_embedding)
        
        optimizer.zero_grad()

        loss = model.loss_function(
            image_embedding, party, decoded_image, mu, logvar, classifier_result
        )

        loss.backward()

        train_loss += loss.item()
        optimizer.step()
        
        # print(party)
        # print(classifier_result)
        # print(torch.argmax(classifier_result, dim=-1))
        
        correct += (party.to(device) == torch.argmax(classifier_result, dim=-1)).float().sum()

    print(f"Train Epoch: {epoch+1} \tAccuracy: {(correct * 100) / len(train_dataset):.2f}% \tLoss: {train_loss / len(train_dataloader):.6f}")


Using cuda device
Train Epoch: 1 	Accuracy: 57.46% 	Loss: 5.805368
Train Epoch: 2 	Accuracy: 57.53% 	Loss: 5.803465
Train Epoch: 3 	Accuracy: 57.53% 	Loss: 5.803390
Train Epoch: 4 	Accuracy: 57.53% 	Loss: 5.803332
Train Epoch: 5 	Accuracy: 57.53% 	Loss: 5.803265
Train Epoch: 6 	Accuracy: 57.52% 	Loss: 5.803319
Train Epoch: 7 	Accuracy: 57.53% 	Loss: 5.803260
Train Epoch: 8 	Accuracy: 57.53% 	Loss: 5.803303
Train Epoch: 9 	Accuracy: 57.53% 	Loss: 5.803213
Train Epoch: 10 	Accuracy: 57.53% 	Loss: 5.803238


In [23]:
# Test Model

model.eval()
test_loss = 0
test_correct = 0

for batch_idx, (image_embedding, party) in enumerate(test_dataloader):
    image_embedding = image_embedding.to(device)

    decoded_image, mu, logvar, classifier_result = model(image_embedding)

    optimizer.zero_grad()

    loss = model.loss_function(
        image_embedding, party, decoded_image, mu, logvar, classifier_result
    )

    loss.backward()

    test_loss += loss.item()
    optimizer.step()

    test_correct += (party.to(device) == torch.argmax(classifier_result, dim=-1)).float().sum()

print(f"Test Metrics \tAccuracy: {(test_correct * 100) / len(test_dataset):.2f}% \tLoss: {test_loss / len(test_dataloader):.6f}")


Test Metrics 	Accuracy: 57.97% 	Loss: 5.801511
